In [1]:
from keras.applications import vgg16
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

from keras.models import load_model
import numpy as np 

Using TensorFlow backend.


In [2]:
model = vgg16.VGG16(weights='imagenet', include_top  = False, input_shape=(224,224,3))

In [3]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [4]:
for layer in model.layers:
    layer.trainable = False

In [5]:
model.layers

In [6]:
top = model.output

In [7]:
top = Flatten()(top)

In [8]:
top = Dense(1024, activation='relu')(top)

In [9]:
top = Dense(512, activation='relu')(top)

In [10]:
top = Dense(1, activation='sigmoid')(top)

In [11]:
newmodel = Model(inputs = model.input, outputs = top)

In [12]:
newmodel.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [13]:
newmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        'dataset/training/',
        target_size=(224,224),
        batch_size=1,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        'dataset/testing/',
        target_size=(224,224),
        batch_size=1,
        class_mode='binary')
newmodel.fit(
        training_set,
        epochs=2,
        validation_data=test_set)

Found 201 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/2
201/201 [==============================] - 147s 730ms/step - loss: 1.1003 - accuracy: 0.8806 - val_loss: 5.0295e-07 - val_accuracy: 1.0000
Epoch 2/2
201/201 [==============================] - 143s 710ms/step - loss: 0.0569 - accuracy: 0.9900 - val_loss: 0.0012 - val_accuracy: 1.0000


In [15]:
r = training_set.class_indices

test_image = image.load_img('dataset/testing/5.jpg', 
               target_size=(224,224))

In [16]:
r

{'cats': 0, 'daksh': 1}

In [17]:
test_image = image.img_to_array(test_image)

In [18]:
test_image.shape

(224, 224, 3)

In [19]:
test_image = np.expand_dims(test_image, axis=0)

In [20]:
result = newmodel.predict(test_image)

In [21]:
result

array([[1.]], dtype=float32)

In [22]:
if result == 1.0:
    print('daksh')
else:
    print('NA')

daksh
